# Utils

Cells in this notebook can be used to perform scriptable tasks like generating pages based on data structures. Normally these would be implemented as Jekyll plugins, but GitHub Pages does not allow custom plugins. There's probably a more clever way to run these tasks in GitHub workflows or something, but this works fine.

## Generate Tag Pages

The following cell reads `_config.yml` and all collection files, extracts tags, and writes a `.md` file for each tag under `tags/` as `tags/<tag>.md`.

Run this cell whenever new tags are added.


In [26]:
import os
import re
import glob
import yaml

# Load Jekyll configuration
with open('_config.yml', 'r') as f:
    cfg = yaml.safe_load(f)

# Determine collections (including posts)
collections = set(cfg.get('collections', {}).keys())
collections.add('posts')

# Gather tags from all collection files
tags = set()
for coll in collections:
    dir_name = '_posts' if coll == 'posts' else f"_{coll}"
    if not os.path.isdir(dir_name):
        continue
    for path in glob.glob(os.path.join(dir_name, '*.md')):
        with open(path, 'r') as f:
            content = f.read()
        fm = re.match(r'^---\s*(.*?)\s*---', content, re.DOTALL)
        if not fm:
            continue
        data = yaml.safe_load(fm.group(1))
        for t in data.get('tags', []) or []:
            if isinstance(t, str):
                t = t.strip()
                tags.add(t)

# Create tags directory and write tag pages
os.makedirs('_tags', exist_ok=True)
for tag in tags:
    slug = re.sub(r'[^\w-]', '', tag.lower().replace(' ', '-'))
    header = f"""---
layout: tag
tag: {tag}
title: {tag}
---"""
    with open(os.path.join('_tags', f'{slug}.md'), 'w') as f:
        f.write(header + '\n')
print(f"Generated {len(tags)} tag pages in '_tags/' directory.")

Generated 91 tag pages in '_tags/' directory.


## Generate Markdown File for Each Image File

This cell generates a corresponding markdown file for each image in a directory (drawings or stereoscopic-images etc.) and puts it in the corresponding Jekyll collection directory with the same name, just with the underscore prefix. If the markdown file already exists, it will _not_ be overwritten.

In [1]:
import os
import re
import sys
from pathlib import Path

def generate_md_for_images(directory, dest_dir, layout):
    img_pattern = re.compile(r'^(?P<date>\d{4}-\d{2}-\d{2})_(?P<name>.+)\.(?:jpg|png|gif)$', re.IGNORECASE)
    directory = Path(directory)
    # if the source directory doesn't exist, exit
    if not directory.exists():
        print(f"Source directory {directory} does not exist.")
        return
    dest_dir = Path(dest_dir)
    # if the destination directory doesn't exist, create it
    if not dest_dir.exists():
        dest_dir.mkdir(parents=True)
    for img_path in directory.iterdir():
        if not img_path.is_file():
            continue
        m = img_pattern.match(img_path.name)
        if not m:
            continue
        date = m.group('date')
        name = m.group('name')
        filename = img_path.name
        pagename = f"{date}_{name}"
        md_path = dest_dir / f"{pagename}.md"
        if md_path.exists():
            continue
        front_matter = f"""---
layout: {layout}
filename: {filename}
pagename: {pagename}
date: {date}
tags:
---\n"""
        md_path.write_text(front_matter)
        print(f"Created {md_path}")

directory = 'drawings/'
dest_dir = '_drawings/'
layout = 'drawing'
generate_md_for_images(directory, dest_dir, layout)

directory = 'stereoscopic-images/'
dest_dir = '_stereoscopic_images/'
layout = 'stereoscopic_image'
generate_md_for_images(directory, dest_dir, layout)


Created _drawings/2025-06-18_third-eye.md
Created _drawings/2025-06-20_octograms.md
Created _drawings/2025-06-23_cycle.md
Created _drawings/2025-06-21_frayed-seams.md
Created _drawings/2025-06-19_face-box-box-face.md
Created _drawings/2025-06-22_infinity-worm.md
Created _stereoscopic_images/2025-06-24_friends.md
Created _stereoscopic_images/2025-06-22_forest.md
Created _stereoscopic_images/2025-06-21_steps.md
Created _stereoscopic_images/2025-06-20_rocks.md
Created _stereoscopic_images/2025-06-19_trees.md
Created _stereoscopic_images/2025-06-21_stage.md
Created _stereoscopic_images/2025-06-23_spurge.md


In [17]:
# find images with the name pattern YYYY-MM-DD.jpg and rename them to YYYY-MM-DD_1.jpg, YYYY-MM-DD_2.jpg, etc.
def rename_images(directory):
    # pattern is YYYY-MM-DD.jpg
    img_pattern = re.compile(r'^(?P<date>\d{4}-\d{2}-\d{2})\.jpg$', re.IGNORECASE)
    directory = Path(directory)
    for img_path in directory.iterdir():
        if not img_path.is_file():
            continue
        m = img_pattern.match(img_path.name)
        if not m:
            continue
        date = m.group('date')
        name = '1'
        new_name = f"{date}_{name}.jpg"
        new_path = directory / new_name
        if new_path.exists():
            continue
        img_path.rename(new_path)
        print(f"Renamed {img_path} to {new_path}")

rename_images('stereoscopic-images/')
rename_images('drawings/')

Renamed drawings/2024-11-21.jpg to drawings/2024-11-21_1.jpg
Renamed drawings/2024-11-20.jpg to drawings/2024-11-20_1.jpg
Renamed drawings/2024-11-22.jpg to drawings/2024-11-22_1.jpg
Renamed drawings/2024-11-23.jpg to drawings/2024-11-23_1.jpg
Renamed drawings/2024-11-18.jpg to drawings/2024-11-18_1.jpg
Renamed drawings/2024-11-24.jpg to drawings/2024-11-24_1.jpg
Renamed drawings/2024-11-25.jpg to drawings/2024-11-25_1.jpg
Renamed drawings/2024-11-19.jpg to drawings/2024-11-19_1.jpg
Renamed drawings/2024-11-17.jpg to drawings/2024-11-17_1.jpg
Renamed drawings/2024-11-16.jpg to drawings/2024-11-16_1.jpg
